In [1]:
import cv2
import numpy as np
import os

# Define constants
haar_file = 'haarcascade_frontalface_default.xml'
datasets = r'C:\Users\aanki\Anti spooling\datasets'
(width, height) = (130, 100)
motion_threshold = 1000  # Adjust this threshold based on your testing
real_detection_threshold = 5  # Number of detections required to confirm as Real

# Function to create a dataset
def create_dataset(sub_data):
    path = os.path.join(datasets, sub_data)
    if not os.path.isdir(path):
        os.makedirs(path)

    face_cascade = cv2.CascadeClassifier(haar_file)
    webcam = cv2.VideoCapture(0)

    count = 1
    while count <= 30:
        (_, im) = webcam.read()
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 4)
        for (x, y, w, h) in faces:
            cv2.rectangle(im, (x, y), (x + w, y + h), (255, 0, 0), 2)
            face = gray[y:y + h, x:x + w]
            face_resize = cv2.resize(face, (width, height))
            cv2.imwrite(f'{path}/{count}.png', face_resize)
            count += 1
            if count > 30:
                break

        cv2.imshow('OpenCV', im)
        key = cv2.waitKey(10)
        if key == 27:
            break

    webcam.release()
    cv2.destroyAllWindows()

# Function to recognize faces
def recognize_faces():
    print('Recognizing Face. Please be in sufficient light...')

    # Load the dataset
    (images, labels, names, id) = ([], [], {}, 0)
    for (subdirs, dirs, files) in os.walk(datasets):
        for subdir in dirs:
            names[id] = subdir
            subjectpath = os.path.join(datasets, subdir)
            for filename in os.listdir(subjectpath):
                path = subjectpath + '/' + filename
                img = cv2.imread(path, 0)
                if img is not None:
                    face_resize = cv2.resize(img, (width, height))
                    images.append(face_resize)
                    labels.append(int(id))
            id += 1

    images = np.array(images)
    labels = np.array(labels)

    # Train the model
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(images, labels)

    face_cascade = cv2.CascadeClassifier(haar_file)
    webcam = cv2.VideoCapture(0)

    ret, prev_frame = webcam.read()
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    real_detection_count = 0  # To count detections of a real registered user
    detection_log = {}  # To keep track of detections for each face

    while True:
        ret, frame = webcam.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Check for motion
        diff_frame = cv2.absdiff(prev_gray, gray)
        _, thresh_frame = cv2.threshold(diff_frame, 25, 255, cv2.THRESH_BINARY)
        motion = np.sum(thresh_frame)

        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            face = gray[y:y + h, x:x + w]
            face_resize = cv2.resize(face, (width, height))
            prediction = model.predict(face_resize)

            name = "Unknown"
            if prediction[1] < 100:
                name = names[prediction[0]]
                cv2.putText(frame, f'{name} - {prediction[1]:.0f}', (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
                print(f"User recognized: {name} (Confidence: {prediction[1]:.0f}) - Registered")
                
                # Track detections for the detected user
                if prediction[0] not in detection_log:
                    detection_log[prediction[0]] = {'count': 0, 'name': name}
                detection_log[prediction[0]]['count'] += 1

                # Confirm as Real if the detection count exceeds threshold
                if detection_log[prediction[0]]['count'] >= real_detection_threshold:
                    real_detection_count += 1
                    if real_detection_count > 5:
                        cv2.putText(frame, 'Live Person Confirmed', (x - 10, y + h + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
                        print("Live person confirmed.")
                        webcam.release()
                        cv2.destroyAllWindows()
                        return

            else:
                cv2.putText(frame, 'Unregistered Face', (x - 10, y - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))
                print("User recognized: Unregistered Face - Fake")

            # Liveness detection
            if motion < motion_threshold:
                cv2.putText(frame, 'Static Image Detected', (x - 10, y + h + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))
                print("Static Image Detected")
            else:
                cv2.putText(frame, 'Live Person', (x - 10, y + h + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0))
                print("Live Person Detected")

        cv2.imshow('OpenCV', frame)
        prev_gray = gray

        key = cv2.waitKey(10)
        if key == 27:
            break

    webcam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    mode = input("Enter 'r' to register a new user or 't' to test recognition: ").strip().lower()
    if mode == 'r':
        user_name = input("Enter the name for registration: ").strip()
        create_dataset(user_name)
    elif mode == 't':
        recognize_faces()
    else:
        print("Invalid mode selected!")

Enter 'r' to register a new user or 't' to test recognition:  t


Recognizing Face. Please be in sufficient light...
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Live Person Detected
User recognized: Unregistered Face - Fake
Static Image Detected
User recognized: Unregistered Face - Fake
Live Person Detected
Use